## Atributos generados

In [ ]:
execID=21
algorithm = "RandomForestCm"
version= "1.0"

## Parámetros comunes

In [ ]:
products = ['LS8_OLI_LASRC','LS7_ETM_LEDAPS' ] #Productos sobre los que se hará la consulta (unidades de almacenamiento)
bands=["blue","green","red","nir", "swir1","swir2"] #arreglo de bandas 
time_ranges = [("2016-01-01", "2016-12-31")] #Una lista de tuplas, cada tupla representa un periodo
#área sobre la cual se hará la consulta:
min_long = -75
min_lat = 10
max_long = -74
max_lat = 11

## Parámetros específicos del algoritmo

In [ ]:
train_data_path = '/home/cubo/notebooks/Piloto_Palma/train-piloto'
validation_data_path= '/home/cubo/notebooks/Piloto_Palma/test-piloto'
normalized=True
minValid=1;

# Librerías

In [ ]:
import datacube
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
import os
import re
import xarray as xr
import numpy as np
import gdal
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score

In [ ]:
def enmascarar_entrenamiento(vector_data_path, cols, rows, geo_transform, projection, target_value=1):
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds

In [ ]:
def rasterizar_entrenamiento(file_paths, rows, cols, geo_transform, projection):
    labeled_pixels = np.zeros((rows, cols))
    for i, path in enumerate(file_paths):
        label = i+1
        ds = enmascarar_entrenamiento(path, cols, rows, geo_transform, projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels

In [ ]:
def exportar(fname, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None

# Consulta

Consulta sobre las diferentes unidades y aplica la máscara de nubes adecuada. 

In [ ]:
nodata=-9999
#Definir las funciones necesatias para el algoritmo
def isin(element, test_elements, assume_unique=False, invert=False):
    "definiendo la función isin de numpy para la versión anterior a la 1.13, en la que no existe"
    element = np.asarray(element)
    return np.in1d(element, test_elements, assume_unique=assume_unique,
                invert=invert).reshape(element.shape)

### Máscara de nubes

Con el nuevo formato, los valores de `pixel_qa` dependen del producto. Para crear la máscara de nubes, se determinan los valores válidos para el producto actual y se usa la banda `pixel_qa` para generar un arreglo de datos booleanos: Para cada posición, si el valor de pixel_qa está en la lista de valores válidos será `True`, en caso contrario será `False`.

In [ ]:
kwargs={}
dc = datacube.Datacube(app="{}_{}_{}".format(algorithm,version,execID))
for product in products:
    i=0
    validValues=set()
    if product=="LS7_ETM_LEDAPS":
        validValues=[66,68,130,132]
    elif product == "LS8_OLI_LASRC":
        validValues=[322, 386, 834, 898, 1346, 324, 388, 836, 900, 1348]
    for tr in time_ranges:
        _data = dc.load(product=product, longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=tr)
        if len(_data.data_vars)==0:
            break
        cloud_mask=isin(_data["pixel_qa"].values, validValues)
        for band in bands:
            _data[band].values=np.where(np.logical_and(_data.data_vars[band]!=nodata,cloud_mask),_data.data_vars[band], np.nan)
        _undesired=list(set(_data.keys())-set(bands+['latitude','longitude','time']))
        _data=_data.drop(_undesired)
            
        if "xarr"+str(i) in kwargs:
            kwargs["xarr"+str(i)]=xr.concat([kwargs["xarr"+str(i)],_data.copy(deep=True)], 'time')
        else:
            kwargs["xarr"+str(i)]=_data
    i+=1
del _data

In [ ]:
_undesired

In [ ]:
#El algoritmo recibe los productos como xarrays en variablles llamadas xarr0, xarr1, xarr2... 
xarr0=kwargs["xarr0"]
del kwargs

In [ ]:
xarr0

# Procesamiento

El algoritmo debe ser auto contenido. Puede importar y usar las librerías disponibles en CDCol, por ejemplo: 

- scikit-learn
- numpy
- xarray
- pycurl
- pandas
- nltk

In [ ]:
dc = datacube.Datacube(app="{}_{}_{}".format(algorithm,version,execID))

_data = dc.load(product="FNF_COL_UTM", longitude=(min_long, max_long), latitude=(min_lat, max_lat), time=tr)

In [ ]:
_data["fnf_mask"].values[0]==1

## Compuesto temporal de medianas

In [ ]:
medians={} 
for band in bands:
    datos=xarr0[band].values
    allNan=~np.isnan(datos) #Una mascara que indica qué datos son o no nan. 
    if normalized: #Normalizar, si es necesario.
        #Para cada momento en el tiempo obtener el promedio y la desviación estándar de los valores de reflectancia
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        # usar ((x-x̄)/st) para llevar la distribución a media 0 y desviación estándar 1, 
        # y luego hacer un cambio de espacio para la nueva desviación y media. 
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    #Calcular la mediana en la dimensión de tiempo 
    medians[band]=np.nanmedian(datos,0) 
    #Eliminar los valores que no cumplen con el número mínimo de pixeles válidos dado. 
    medians[band][np.sum(allNan,0)<minValid]=np.nan
    medians[band][_data["fnf_mask"].values[0]==1]=np.nan
del datos
del _data

In [ ]:
_coords=xarr0.coords
_crs=xarr0.crs
del xarr0

In [ ]:
files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
shapefiles = [os.path.join(train_data_path, f) for f in files if f.endswith('.shp')]

In [ ]:
shapefiles

In [ ]:
rows, cols = medians[bands[0]].shape

In [ ]:
#(originX, pixelWidth, 0, originY, 0, pixelHeight)
geo_transform=(_coords["longitude"].values[0], 0.000269995,0, _coords["latitude"].values[0],0,-0.000271302)

In [ ]:
proj=_crs.wkt
#proj='GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [ ]:
labeled_pixels = rasterizar_entrenamiento(shapefiles, rows, cols, geo_transform, proj)

In [ ]:
labeled_pixels

In [ ]:
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
nmed=None
bands_data=[]
for band in bands: 
    bands_data.append(medians[band])
bands_data = np.dstack(bands_data)
#training_samples = nmed[is_train]


In [ ]:
rows, cols, n_bands = bands_data.shape

In [ ]:
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
training_samples = bands_data[is_train]

In [ ]:
_msk=np.sum(np.isfinite(training_samples),1)>0
training_samples= training_samples[_msk,:]
training_labels=training_labels[_msk]

In [ ]:
classifier = RandomForestClassifier(n_jobs=-1, n_estimators=250, verbose=1)
classifier.fit(training_samples, training_labels)

In [ ]:
from sklearn.externals import joblib
joblib.dump(classifier,'modelo2.pkl')

In [ ]:
n_samples = rows*cols
flat_pixels = bands_data.reshape((n_samples, n_bands))

In [ ]:
_msk=np.sum(np.isfinite(flat_pixels),1)>0
print _msk.shape

In [ ]:
result = classifier.predict(flat_pixels[_msk])
a=np.empty(rows*cols)
a[:]=np.nan
a[_msk]=result
classification = a.reshape((rows, cols))

In [ ]:
#exportar("salida-class.tiff", classification, geo_transform, proj)

## Preparar la salida
La salida de los algoritmos puede expresarse como: 
- un xarray llamado output (que debe incluir entre sus atributos el crs del sistema de coordenadas)
- un diccionario con varios xarray llamado `outputs`
- Texto, en una variable llamada `outputtxt`

In [ ]:
import xarray as xr
ncoords=[]
xdims =[]
xcords={}
for x in _coords:
    if(x!='time'):
        ncoords.append( ( x, _coords[x]) )
        xdims.append(x)
        xcords[x]=_coords[x]
        
variables={}
#variables ={k: xr.DataArray(v, dims=xdims,coords=ncoords)
#             for k, v in medians.items()}
variables["classification"]=xr.DataArray(classification,dims=xdims,coords=ncoords)
output=xr.Dataset(variables, attrs={'crs':_crs})

for x in output.coords:
    _coords[x].attrs["units"]=_coords[x].units

# Guardar la salida
La tarea genérica se encarga de generar los archivos de salida en la carpeta adecuada. 

__Nota__: A diferencia de la tarea genérica, que maneja los 3 tipos de salida descritos en la sección anterior, este cuaderno sólo guarda la salida definida en output

In [ ]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
print "{}_{}_{}.nc".format(algorithm,version,execID)
nco=netcdf_writer.create_netcdf("{}_{}_{}.nc".format(algorithm,version,execID))
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, _coords[x].values, _coords[x].units)
netcdf_writer.create_grid_mapping_variable(nco, _crs)
var= netcdf_writer.create_variable(nco, "classification", Variable(np.dtype(np.int32), None, ('latitude', 'longitude'), None) ,set_crs=True)
var[:] = netcdf_writer.netcdfy_data(classification)
nco.close()